In [1]:
import pandas as pd
import requests

In [2]:
response = requests.get('https://api.opendota.com/api/heroStats/')
data = response.json()

In [3]:
heroes = pd.json_normalize(data).drop(['name', 'icon', 'img', 'base_health', 'null_win', 'legs', 'id', 'pro_ban', 
                                       'turn_rate', '1_win', '1_pick', '2_pick', '2_win', '3_pick', '3_win', '4_pick', 
                                       '4_win', '5_pick', '5_win', '6_pick', '6_win', '7_pick', '7_win', 'null_pick', 
                                       'turbo_picks', 'turbo_wins', 'base_mana', 'base_mr', 'cm_enabled'], axis=1)

In [4]:
#heroes = pd.read_csv('heroes.csv')
df = pd.read_csv('C:/Users/Admin/Python/EDUCATION/DOTA/Data/DataFrame/maindf.csv')

# 1. Предобработка данных

## 1.1 Матчи

In [5]:
# Последние два патча
df = df[df['patch'].isin([50, 51])]

In [6]:
df = df[['radiant_picks', 'dire_picks', 'radiant_win']]

In [7]:
df['radiant_picks'] = df['radiant_picks'].apply(eval)
df['dire_picks'] =  df['dire_picks'].apply(eval)

In [8]:
df.head()

,radiant_picks,dire_picks,radiant_win
0,"[3, 108, 74, 9, 8]","[81, 106, 21, 128, 49]",True
1,"[101, 129, 23, 3, 12]","[128, 136, 25, 94, 14]",True
2,"[114, 13, 3, 81, 53]","[47, 136, 23, 110, 129]",False
3,"[81, 128, 105, 106, 54]","[136, 109, 121, 19, 38]",True
4,"[81, 111, 90, 106, 76]","[16, 101, 85, 94, 92]",False


## 1.2 Герои

In [9]:
heroes['base_attack_avg'] = (heroes['base_attack_min'] + heroes['base_attack_max']) / 2

In [10]:
heroes.index = heroes.hero_id
heroes.drop(['hero_id', 'base_attack_min', 'base_attack_max'], axis=1, inplace=True)

In [11]:
heroes.head(3)

,localized_name,primary_attr,attack_type,roles,base_health_regen,base_mana_regen,base_armor,base_str,base_agi,base_int,...,int_gain,attack_range,projectile_speed,attack_rate,move_speed,pro_win,pro_pick,8_pick,8_win,base_attack_avg
hero_id,,,,,,,,,,,,,,,,,,,,,
1,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",0.25,0.0,0.0,21,24,12,...,1.8,150,0,1.4,310,18,38,1008,500,31.0
2,Axe,str,Melee,"[Initiator, Durable, Disabler, Jungler, Carry]",2.50,0.0,-1.0,25,20,18,...,1.6,150,900,1.7,310,28,63,1594,737,29.0
3,Bane,int,Ranged,"[Support, Disabler, Nuker, Durable]",0.25,0.0,1.0,22,22,22,...,2.5,400,900,1.7,305,69,122,1084,568,36.0


In [54]:
# Герои с крайним значением винрейта
worst_wr = (heroes['pro_win'] / heroes['pro_pick']).sort_values().head(15).index.tolist()
best_wr = (heroes['pro_win'] / heroes['pro_pick']).sort_values().tail(15).index.tolist()

# 2. Объединение датасетов

In [55]:
def pick_to_features(lst, prefix='r_'):
    res = dict()
    for i in lst:    
        new_hero = heroes.loc[i]
        res[prefix + new_hero['primary_attr']] = res.get(prefix + new_hero['primary_attr'], 0) + 1
        res[prefix + new_hero['attack_type']] = res.get(prefix + new_hero['attack_type'], 0) + 1
        for role in new_hero['roles']:
            res[prefix + role] = res.get(prefix + role, 0) + 1
    
        res[prefix + 'sum_base_health_regen'] = res.get(prefix + 'sum_base_health_regen', 0) + new_hero['base_health_regen']
        res[prefix + 'sum_base_mana_regen'] = res.get(prefix + 'sum_base_mana_regen', 0) + new_hero['base_mana_regen']
        res[prefix + 'sum_base_armor'] = res.get(prefix + 'sum_base_armor', 0) + new_hero['base_armor']
        res[prefix + 'sum_base_str'] = res.get(prefix + 'sum_base_str', 0) + new_hero['base_str']
        res[prefix + 'sum_base_agi'] = res.get(prefix + 'sum_base_agi', 0) + new_hero['base_agi']
        res[prefix + 'sum_base_int'] = res.get(prefix + 'sum_base_int', 0) + new_hero['base_int']
        res[prefix + 'sum_str_gain'] = res.get(prefix + 'sum_str_gain', 0) + new_hero['str_gain']
        res[prefix + 'sum_agi_gain'] = res.get(prefix + 'sum_agi_gain', 0) + new_hero['agi_gain']
        res[prefix + 'sum_int_gain'] = res.get(prefix + 'sum_int_gain', 0) + new_hero['int_gain']
        res[prefix + 'sum_attack_range'] = res.get(prefix + 'sum_attack_range', 0) + new_hero['attack_range']
        res[prefix + 'sum_projectile_speed'] = res.get(prefix + 'sum_projectile_speed', 0) + new_hero['projectile_speed']
        res[prefix + 'sum_attack_rate'] = res.get(prefix + 'sum_attack_rate', 0) + new_hero['attack_rate']
        res[prefix + 'sum_move_speed'] = res.get(prefix + 'sum_move_speed', 0) + new_hero['move_speed']
        res[prefix + 'sum_base_attack_avg'] = res.get(prefix + 'sum_base_attack_avg', 0) + new_hero['base_attack_avg']
        res[prefix + 'sum_pro_win'] = res.get(prefix + 'sum_pro_win', 0) + new_hero['pro_win']
        res[prefix + 'sum_pro_pick'] = res.get(prefix + 'sum_pro_pick', 0) + new_hero['pro_pick']
        res[prefix + 'sum_8_win'] = res.get(prefix + 'sum_8_win', 0) + new_hero['8_win']
        res[prefix + 'sum_8_pick'] = res.get(prefix + 'sum_8_pick', 0) + new_hero['8_pick']    
    return res

In [56]:
rad_df = pd.json_normalize(df['radiant_picks'].apply(pick_to_features))

In [57]:
dire_df = pd.json_normalize(df['dire_picks'].apply(pick_to_features, prefix='d_'))

In [58]:
rad_df.fillna(0, inplace=True)
dire_df.fillna(0, inplace=True)

- признаки Ranged и Melee - противоположности - достаточно использовать один
- Каждый герой имеет только один атрибут (int, str, agi). Зная два признака из нашего датасета, можно определить третий(Их сумма ровна 0). Поэтому не будем использовать признак agi

In [59]:
pickdf  = pd.DataFrame()

In [60]:
# Роли
pickdf['support'] = rad_df['r_Support'] - dire_df['d_Support']
pickdf['disabler'] = rad_df['r_Disabler'] - dire_df['d_Disabler']
pickdf['nuker'] = rad_df['r_Nuker'] - dire_df['d_Nuker']
pickdf['durable'] = rad_df['r_Durable'] - dire_df['d_Durable']
pickdf['escape'] = rad_df['r_Escape'] - dire_df['d_Escape']
pickdf['carry'] = rad_df['r_Carry'] - dire_df['d_Carry']
pickdf['pusher'] = rad_df['r_Pusher'] - dire_df['d_Pusher']
pickdf['initiator'] = rad_df['r_Initiator'] - dire_df['d_Initiator']
pickdf['jungler'] = rad_df['r_Jungler'] - dire_df['d_Jungler']

# Тип атаки
pickdf['ranged'] = rad_df['r_Ranged'] - dire_df['d_Ranged']

# Атрибут
pickdf['int'] = rad_df['r_int'] - dire_df['d_int']
pickdf['str'] = rad_df['r_str'] - dire_df['d_str']

# Cтаты героев
pickdf['sum_base_int'] = rad_df['r_sum_base_int'] - dire_df['d_sum_base_int']
pickdf['sum_base_str'] = rad_df['r_sum_base_str'] - dire_df['d_sum_base_str']
pickdf['sum_base_agi'] = rad_df['r_sum_base_agi'] - dire_df['d_sum_base_agi']
pickdf['sum_int_gain'] = rad_df['r_sum_int_gain'] - dire_df['d_sum_int_gain']
pickdf['sum_str_gain'] = rad_df['r_sum_str_gain'] - dire_df['d_sum_str_gain']
pickdf['sum_agi_gain'] = rad_df['r_sum_agi_gain'] - dire_df['d_sum_agi_gain']

pickdf['sum_attack_range'] = rad_df['r_sum_attack_range'] - dire_df['d_sum_attack_range']
pickdf['sum_attack_rate'] = rad_df['r_sum_attack_rate'] - dire_df['d_sum_attack_rate']
pickdf['sum_attack_rate'] = rad_df['r_sum_attack_rate'] - dire_df['d_sum_attack_rate']
pickdf['sum_base_attack_avg'] = rad_df['r_sum_base_attack_avg'] - dire_df['d_sum_base_attack_avg']
pickdf['sum_projectile_speed'] = rad_df['r_sum_projectile_speed'] - dire_df['d_sum_projectile_speed']
pickdf['sum_move_speed'] = rad_df['r_sum_move_speed'] - dire_df['d_sum_move_speed']

pickdf['sum_base_health_regen'] = rad_df['r_sum_base_health_regen'] - dire_df['d_sum_base_health_regen']
pickdf['sum_base_mana_regen'] = rad_df['r_sum_base_mana_regen'] - dire_df['d_sum_base_mana_regen']
pickdf['sum_base_armor'] = rad_df['r_sum_base_armor'] - dire_df['d_sum_base_armor']

# Винрейт
pickdf['pro_wr_att'] = (rad_df['r_sum_pro_win'] / rad_df['r_sum_pro_pick']) / \
                       (dire_df['d_sum_pro_win'] / dire_df['d_sum_pro_pick'])
pickdf['imm_wr_att'] = (rad_df['r_sum_8_win'] / rad_df['r_sum_8_pick']) / \
                       (dire_df['d_sum_8_win'] / dire_df['d_sum_8_pick'])

In [61]:
def pick_to_best(lst):
    return len(set(lst).intersection(set(best_wr)))

def pick_to_worst(lst):
    return len(set(lst).intersection(set(worst_wr)))

In [62]:
pickdf['best_picks'] = df['radiant_picks'].apply(pick_to_best) - df['dire_picks'].apply(pick_to_best)
pickdf['worst_picks'] = df['radiant_picks'].apply(pick_to_worst) - df['dire_picks'].apply(pick_to_worst)

In [63]:
pickdf['radiant_win'] = df['radiant_win'].apply('int64')

In [64]:
pickdf.head()

,support,disabler,nuker,durable,escape,carry,pusher,initiator,jungler,ranged,...,sum_projectile_speed,sum_move_speed,sum_base_health_regen,sum_base_mana_regen,sum_base_armor,pro_wr_att,imm_wr_att,best_picks,worst_picks,radiant_win
0,1.0,-1.0,0.0,0.0,1.0,-1.0,0.0,-3.0,0.0,1.0,...,-2150,-55,-1.25,0.40,1.0,1.021109,1.021074,0.0,0.0,1
1,0.0,-1.0,1.0,1.0,-1.0,0.0,1.0,0.0,0.0,-1.0,...,600,90,0.00,0.50,0.0,1.007278,1.011806,0.0,0.0,1
2,-2.0,-1.0,1.0,-1.0,1.0,-1.0,2.0,-2.0,1.0,1.0,...,625,25,-1.25,0.75,4.0,0.997198,1.007734,0.0,0.0,0
3,-1.0,1.0,-1.0,0.0,1.0,0.0,-1.0,-1.0,1.0,1.0,...,2700,30,-1.00,1.00,1.0,0.922844,0.961975,-1.0,0.0,1
4,-2.0,0.0,0.0,-2.0,1.0,2.0,0.0,1.0,0.0,0.0,...,-900,85,1.75,0.25,7.5,0.980042,0.947180,1.0,-1.0,0


In [65]:
pickdf.corr(method='spearman')['radiant_win']

support                 -0.012747
disabler                -0.014786
nuker                    0.007275
durable                  0.012350
escape                  -0.015428
carry                   -0.021374
pusher                   0.040897
initiator                0.010130
jungler                  0.022349
ranged                   0.033870
int                      0.011660
str                     -0.011210
sum_base_int             0.016740
sum_base_str            -0.014353
sum_base_agi            -0.000271
sum_int_gain             0.013925
sum_str_gain            -0.009135
sum_agi_gain             0.000706
sum_attack_range         0.016227
sum_attack_rate         -0.005964
sum_base_attack_avg      0.006798
sum_projectile_speed    -0.022117
sum_move_speed           0.009853
sum_base_health_regen   -0.037788
sum_base_mana_regen     -0.005353
sum_base_armor          -0.008790
pro_wr_att               0.116464
imm_wr_att               0.066592
best_picks               0.015159
worst_picks   

# 3. Обучение

In [66]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [67]:
x_train, x_test, y_train, y_test = train_test_split(pickdf.drop('radiant_win', axis=1), 
                                                    pickdf['radiant_win'], test_size=0.25, random_state=3)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((9333, 30), (9333,), (3111, 30), (3111,))

In [68]:
best_params = {'border_count': 60,
 'depth': 2,
 'l2_leaf_reg': 10,
 'iterations': 400,
 'learning_rate': 0.07}

In [69]:
clf = CatBoostClassifier(**best_params)
clf.fit(x_train, y_train)

0:	learn: 0.6922391	total: 1.34ms	remaining: 534ms
1:	learn: 0.6914116	total: 2.82ms	remaining: 561ms
2:	learn: 0.6907063	total: 4.02ms	remaining: 532ms
3:	learn: 0.6901089	total: 5.3ms	remaining: 525ms
4:	learn: 0.6894586	total: 6.95ms	remaining: 549ms
5:	learn: 0.6889910	total: 8.34ms	remaining: 547ms
6:	learn: 0.6884704	total: 9.71ms	remaining: 545ms
7:	learn: 0.6880820	total: 11.2ms	remaining: 549ms
8:	learn: 0.6877341	total: 12.6ms	remaining: 547ms
9:	learn: 0.6873770	total: 13.9ms	remaining: 541ms
10:	learn: 0.6870232	total: 15.2ms	remaining: 537ms
11:	learn: 0.6867169	total: 16.5ms	remaining: 534ms
12:	learn: 0.6864855	total: 18ms	remaining: 536ms
13:	learn: 0.6861859	total: 19.2ms	remaining: 530ms
14:	learn: 0.6859028	total: 20.5ms	remaining: 527ms
15:	learn: 0.6857297	total: 21.9ms	remaining: 525ms
16:	learn: 0.6855149	total: 23.2ms	remaining: 522ms
17:	learn: 0.6853329	total: 24.6ms	remaining: 522ms
18:	learn: 0.6852157	total: 25.8ms	remaining: 518ms
19:	learn: 0.6851067	tota

In [70]:
clf.score(x_test, y_test)

0.5451623272259724

In [71]:
sorted(clf.feature_importances_.tolist(), reverse=True)[:10]

[19.480238219333906,
 5.948131714427817,
 5.185918883192238,
 5.017005839626123,
 4.905326204405714,
 4.878320403573391,
 4.8112217518250775,
 4.480645517819718,
 4.349694777372652,
 4.0174790366376225]

In [72]:
pd.DataFrame(data=[clf.feature_importances_], columns=x_train.columns)

,support,disabler,nuker,durable,escape,carry,pusher,initiator,jungler,ranged,...,sum_base_attack_avg,sum_projectile_speed,sum_move_speed,sum_base_health_regen,sum_base_mana_regen,sum_base_armor,pro_wr_att,imm_wr_att,best_picks,worst_picks
0,1.978767,0.969769,2.051769,2.747557,1.26209,1.894837,4.480646,3.034,1.2262,1.447626,...,2.471657,2.507611,4.905326,5.017006,4.017479,3.102395,19.480238,5.948132,1.074579,1.539614


# 4. Вывод
- Характеристики героев почти не коррелируют с результатом игры, поэтому их использование нецелесообразно
- Наибольшая корреляция с рейзультатом матча у фичи pro_wr_att